<a href="https://colab.research.google.com/github/arunoda/fastai-v4/blob/master/07_2_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Here are trying to understand what is Embedding is**

## Setting Up on Colab

You only need to run this on Colab.

In [1]:
!pip install fastai2 > /dev/null 2>&1
!git clone https://github.com/arunoda/fastai-v4 > /dev/null 2>&1
%cd fastai-v4

/content/fastai-v4


In [0]:
from fastai2.collab import *
from fastai2.tabular.all import *

## **Loading the Dataset**

Here we are going to use a Mini dataset from Movie Lens.

In [3]:
data_path = untar_data(URLs.ML_100k)

In [0]:
df_ml = pd.read_csv(data_path/"u.data", delimiter = "\t", header=None, names=["user", "movie", "rating", "timestamp"])


In [5]:
df_ml.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [0]:
movies = pd.read_csv(data_path/"u.item", delimiter="|", encoding='latin-1', header=None, names=("movie", "title"), usecols=(0,1))

In [7]:
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


**Now we are going to join these two tables. We do that using the foreign key of `movie`** 

In [8]:
df_ml = df_ml.merge(movies)
df_ml.tail()

,user,movie,rating,timestamp,title
99995,840,1674,4,891211682,Mamma Roma (1962)
99996,655,1640,3,888474646,"Eighth Day, The (1996)"
99997,655,1637,3,888984255,Girls Town (1996)
99998,655,1630,3,887428735,"Silence of the Palace, The (Saimt el Qusur) (1994)"
99999,655,1641,3,887427810,Dadetown (1995)


In [0]:
dls_bs_5 = CollabDataLoaders.from_df(df_ml, item_name="title", bs=5)

In [0]:
x, y = dls_bs_5.one_batch()

In [0]:
dls = CollabDataLoaders.from_df(df_ml, item_name="title", bs=64)

In [14]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_users, n_movies

(944, 1631)

## Create a Model


In [0]:
class EmbedModel(Module):
  def __init__(self, n_users, n_movies, n_factors):
    self.user_factors = Embedding(n_users, n_factors)
    self.movie_factors = Embedding(n_movies, n_factors)

  def forward(self, batch):
    ## See this is why we call embedding. Here we simply call for the index
    ## But it's a mathematic operation which does the same thing as one_hot
    users = self.user_factors(batch[:, 0])
    movies = self.movie_factors(batch[:, 1])

    return (users * movies).sum(dim=1)

In [0]:
model = EmbedModel(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

In [17]:
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,1.313899,1.287353,00:11
1,1.058909,1.081015,00:11
2,0.934283,0.966761,00:11
3,0.812339,0.880719,00:11
4,0.760981,0.866112,00:11


In [0]:
a = nn.Parameter(torch.zeros(4, 2).normal_(0, 0.1))

In [87]:
a

Parameter containing:
tensor([[ 0.0267, -0.1046],
        [-0.0423, -0.0339],
        [ 0.1464, -0.0058],
        [ 0.0271,  0.1022]], requires_grad=True)

**See now it can track to calculate gradients**

In [89]:
a[0]

tensor([ 0.0267, -0.1046], grad_fn=<SelectBackward>)

**See, it uses `SelectBackward`**

Basically, it knows how to calculate gradients even if we select items.

No need to use one_hot

In [0]:
def P(size):
  return nn.Parameter(torch.zeros(*size).normal_(0, 0.1))

In [91]:
P((4,2))

Parameter containing:
tensor([[-0.1476, -0.0275],
        [-0.1326, -0.0248],
        [-0.1310,  0.0143],
        [ 0.0528,  0.1460]], requires_grad=True)

In [0]:
class ParamModel(Module):
  def __init__(self, n_users, n_movies, n_factors):
    self.user_factors = P((n_users, n_factors))
    self.movie_factors = P((n_movies, n_factors))

  def forward(self, batch):
    ## See this is why we call embedding. Here we simply call for the index
    ## But it's a mathematic operation which does the same thing as one_hot
    users = self.user_factors[batch[:, 0]]
    movies = self.movie_factors[batch[:, 1]]

    return (users * movies).sum(dim=1)

In [99]:
model = ParamModel(n_users, n_movies, 4)
model.forward(x)

tensor([-0.0471,  0.0101,  0.0028, -0.0052,  0.0025], grad_fn=<SumBackward1>)

In [0]:
learn = Learner(dls, ParamModel(n_users, n_movies, 50), loss_func=MSELossFlat())

In [107]:
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,2.803879,2.208635,00:11
1,1.009280,1.047862,00:11
2,0.775281,0.950174,00:11
3,0.600679,0.902285,00:11
4,0.524343,0.894039,00:11


In [0]:
learn = Learner(dls, EmbedModel(n_users, n_movies, 50), loss_func=MSELossFlat())

In [109]:
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,1.396997,1.276305,00:11
1,1.064210,1.096655,00:11
2,0.928634,0.999196,00:11
3,0.830301,0.904282,00:11
4,0.800561,0.888393,00:11


**See. It's almost the same thing**